We almost completed our goals. Here are the few things we can complete 
    1. Format this jupyter notebook (we can add mostly main points and conclusions). We mainly need to work on this. her is the colab notebook: https://colab.research.google.com/drive/1GJpz6QQ3cfZVwxEIIa9aacJHIkI2oY2B?usp=sharing
    2. We have so much theory, I am preparing a pdf in overleaf (paper style, to impress them, haha):https://www.overleaf.com/8119818887cpphkvmygwmk
    3. I have prepared a rough presentation. Feel free to modify the slides.https://docs.google.com/presentation/d/1w7aNlGFCkxzRF_F274jCIw3pSWZ4vip-whXrdA4rb5Q/edit?usp=sharing
    
(optional)    
We can also try, 4.Implemetnt the circuit in a real device. \
5. write an introduction to amplitude encoding and randomness

## 1. Introduction

#### How can the utilization of squeezed coherent states in superconducting qubits potentially address the challenges of measurement precision, decoherence, and quantum information transfer, thereby advancing the development of quantum hardware?

## 2. Goals

### (i). Developed a generalized algorithm that incorporates two or more modes of squeezing 
### (ii). Generalize to N-qubit states.
### (iii). Minimize the circuit depth
### (iv). 

In [1]:
#Importing essential libraries

import numpy as np
from numpy import pi
import matplotlib.pyplot as plt

from qiskit import *

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.quantum_info import state_fidelity
from qiskit import Aer, transpile
from qiskit import QuantumCircuit, execute, BasicAer
from qiskit.quantum_info import DensityMatrix
from qiskit.visualization import plot_state_city

## 3. Generating a SPDC reference state

In [2]:
#(write the equation)

In [3]:
def Rn(n, r=1, phi=np.pi):
    '''
    Return the Rn coefficients of the states.
    '''
    return (1/np.cosh(r))*( ( -np.exp(1j*phi)*np.tanh(r) )**n )

In [4]:
def getTargetStateBasic(N, r=1, phi=np.pi):
    '''
    Return a list of the necessary Rn coefficients for a given N.
    
    N: cutoff
    2*N: Number of qbits
    2**N: Number of states
    '''
    Nstates = 2**N
    
    L = [Rn(n, r=r, phi=phi) for n in range(Nstates)]
    L = L / np.linalg.norm(L)
    return L    

In [ ]:
def prettyBin(num, N):
    '''
    Returns *num* as a binary string, zero-padded to N digits.
    ex: prettyBin(3, 6) -> '000011'
    '''
    return '{num:0{N}b}'.format(num=num, N=N)

In [ ]:
def getStateIndex(idx_v1_dec, N):
    '''
    Returns the decimal index of the 'ith state'.
    Ex: N=3, 2nd state -> idx_v1 = '010' -> idx_v2 = '010010' -> idx_v2 = 18
    getStateIndex(2, 3) -> 18
    '''
    idx_v1_bin = prettyBin(idx_v1_dec, N)
    idx_v2_bin = 2 * idx_v1_bin
    idx_v2_dec = int(idx_v2_bin, 2)
    return idx_v2_dec

In [ ]:
def getTargetStateAdvanced(N, r=1, phi=np.pi):
    '''
    Return a list of the necessary Rn coefficients for a given N, but mapped to a system of 2*N qubits.
    
    N: cutoff
    2*N: Number of qbits
    2**N: Number of states
    '''
    
    L = getTargetStateBasic(N, r=r, phi=phi)

    Nqbits = 2*N
    LL = 0j*np.ones(2**Nqbits)

    for idx,val in enumerate(L):
        idx_v2 = getStateIndex(idx, N)
        LL[idx_v2] = val
        
    return LL

In [ ]:
def getReferenceQuantumCircuit(N, r=1, phi=np.pi):
    '''
    Returns a quantum circuit representing the TMSV state with parameters:
    
        * cutoff *N*,
        * squeezing parameter *r*
        * *phi*
    '''
    Nqbits = 2*N
    qc = QuantumCircuit(Nqbits)
    x = getTargetStateAdvanced(N, r=r, phi=phi)
    qc.initialize(x, qubits=range(Nqbits))
    return qc

In [ ]:
def checkQuantumCircuit_v1(qc, N, r=1, phi=np.pi):
    qc_ref = getReferenceQuantumCircuit(N, r=r, phi=phi)

    backend = Aer.get_backend('statevector_simulator')

    sv_ref = execute(qc_ref, backend).result().get_statevector(qc_ref)
    sv = execute(qc, backend).result().get_statevector(qc)

    return state_fidelity(sv_ref, sv)

In [ ]:
def checkQuantumCircuit_v2(qc, N, r=1, phi=np.pi):
    qc_ref = getReferenceQuantumCircuit(N, r=r, phi=phi)
    state_ref = DensityMatrix(qc_ref)
    state = DensityMatrix(qc)
    return state_fidelity(state_ref, state)

In [ ]:
# cutoff
N = 2
# squeezing parameter
r = 1
# phi
phi = np.pi

In [ ]:
### 3(a). 

In [ ]:
Nstates = 2**N
n = range(Nstates)
y = getTargetStateBasic(N, r=r, phi=phi)
#y = Rn(n)
plt.plot(n, np.real(y), 'b-o', label='$real(R_n)$')
#plt.plot(n, np.imag(y), 'r-o', label='$imag(R_n)$')
plt.xlabel('n')
plt.ylabel('$R_n$')
plt.title(f'N={N}, r={r}')
plt.legend()
plt.show()

In [ ]:
# probabilities
P = np.abs(y)**2
plt.plot(n, P, '-o')
plt.xlabel('n')
plt.ylabel('$|R_n|^2$')
plt.title(f'N={N}, r={r}')
plt.show()

In [ ]:
# probabilities

#I am re-defing n as cufoff\
n = np.linspace(1,4,4)

P = np.abs(y)**2
plt.plot(n, P, '-o')
plt.xlabel('Cutoff number')
plt.ylabel('State probability ($|R_n|^2$)')
plt.savefig('ref_state_probability.png', dpi = 1200, transparent =True)
plt.show()

In [ ]:
qc = getReferenceQuantumCircuit(N, r=r, phi=phi)
qc.draw('mpl')

In [ ]:
state = DensityMatrix(qc)
plot_state_city(state, color=['midnightblue', 'crimson'], title="New State City")

In [ ]:
# Calculate fidelity
f1 = checkQuantumCircuit_v1(qc, N, r=r, phi=phi)
f2 = checkQuantumCircuit_v2(qc, N, r=r, phi=phi)
print(f1)
print(f2)

In [ ]:
#Transpiling with the available basic gates
circuit = transpile(qc, basis_gates=['cx', 'id', 'rz', 'x', 'sx'])

In [ ]:
circuit.draw()
plt.savefig('ref_circuit_transpiled.png',dpi = 1200, transparent = True)


In [ ]:
state = DensityMatrix(qc)
plot_state_city(state, color=['midnightblue', 'crimson'], title="New State City")
plt.savefig('ref_circuit_state_Vectors_3d.png',dpi = 1200, transparent = True)


In [ ]:
## Generating an n-qubit Two-Model Squeezed Vacuum (TMSV) state

In [ ]:
##Implementing a n-qubit using Quantum fourier transform

def TMSVQuantumCircuit(N, r=1, phi=np.pi):
    n = 2*N #Number of registers
    qreg_q = QuantumRegister(n, 'q')
    #creg_c = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg_q)


    R = np.tanh(r)#-0.5
    for i in range(N):

        theta =  2*np.arctan(R**(2**i))
        qc.ry(theta, qreg_q[i])
        qc.cx(qreg_q[i], qreg_q[N+i])
    return qc


In [ ]:
## 3. Re

In [ ]:
r = 0.1
phi = np.pi

qc_ref_depth = []
qc_TMSV_depth = []

qc_ref_f1 = []
qc_TMSV_f1 = []

qc_ref_f2 = []
qc_TMSV_f2 = []

Cut_off = np.arange(1,5)

for N in Cut_off:
    print(N)
    qc_ref = getReferenceQuantumCircuit(N, r=r, phi=phi)
    qc_ref = transpile(qc_ref, basis_gates=['cx', 'id', 'rz', 'x', 'sx'])
    
    qc_TMSV = TMSVQuantumCircuit(N, r=r, phi=phi)
    qc_TMSV = transpile(qc_TMSV, basis_gates=['cx', 'id', 'rz', 'x', 'sx'])

    qc_ref_depth.append(qc_ref.depth())
    qc_TMSV_depth.append(qc_TMSV.depth())
    
    f1_ref = checkQuantumCircuit_v1(qc_ref, N, r=r, phi=phi)
    f2_ref = checkQuantumCircuit_v2(qc_ref, N, r=r, phi=phi)

    f1_TMSV = checkQuantumCircuit_v1(qc_TMSV, N, r=r, phi=phi)
    f2_TMSV = checkQuantumCircuit_v2(qc_TMSV, N, r=r, phi=phi)


    qc_ref_f1.append(f1_ref)
    qc_ref_f2.append(f2_ref)

    qc_TMSV_f1.append(f1_TMSV)
    qc_TMSV_f2.append(f2_TMSV)
#qc.measure(qreg_q, creg_c)
#qc.draw()

In [ ]:
#Plotting the circuit depth
plt.plot(Cut_off,qc_ref_depth,'rs-', label='Reference state')
plt.plot(Cut_off,qc_TMSV_depth,'go-', label='Two Mode Squeezed Vacuum (TMSV) ')
plt.plot(Cut_off,qc_TMSV_depth,'g')
plt.xlabel('Cutoff number')
plt.ylabel('Circuit depth')
plt.legend()
plt.savefig('Circuit_depth.png', dpi = 1200, transparent =True)
plt.show()

In [ ]:
print("Number of gates used in the reference state",dict(qc_ref.count_ops()))
print("Number of gates used in the TMSV state",dict(qc_TMSV.count_ops()))

In [ ]:
#Plotting the fidelity of the refernce state and the TMSV state

#plt.plot(qc_ref_f1,'rs-', label='ref')
plt.plot(Cut_off,qc_TMSV_f1,'go-', label='TMSV')
#plt.plot(qc_TMSV_depth,'g')
#plt.ylim([0.999996,1.00001])
plt.xlabel('Cutoff number')
plt.ylabel('State Fidelity')
plt.legend()
plt.savefig('fidelity.png', dpi = 1200, transparent =True)
plt.show()

In [ ]:
#The transpiled circuit of the reference state
qc_ref.draw('mpl')

In [ ]:
#The transpiled circuit of the TMSV state
qc_TMSV.draw('mpl')

In [ ]:
#Running on a quantum computer


In [ ]:
shots = 4096

from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy
if not IBMQ.active_account():
    IBMQ.load_account()
provider = IBMQ.get_provider()
backend = "ibm_nairobi"
backend = least_busy(provider.backends(simulator=False))
print("Least busy backend:", backend.name())

job = execute(qc, backend=backend, shots=shots)

from qiskit.tools.monitor import job_monitor
job_monitor(job)

result = job.result()
plot_histogram(result.get_counts(qc))

